In [ ]:
import datetime
import os
import random
import time

import pandas as pd
import requests
from bs4 import BeautifulSoup

target_data_dir = "../data/1_raw"
os.makedirs(target_data_dir, exist_ok=True)

In [ ]:
def scrape(website: str, target_data_count: int) -> list[str] | None:
    """
    Scrape un nombre donné de titres d'articles depuis un site donné.

    Args:
        website (str): Le nom du site ("legorafi", "nordpresse", "lefigaro", "libération")
        target_data_count (int): Nombre minimal de titres à collecter

    Returns:
        list[str]: Liste de chaînes de caractères contenant les titres
    """
    if website == "legorafi":
        base_url = "https://www.legorafi.fr/"
    elif website == "nordpresse":
        base_url = "https://nordpresse.be/"
    elif website == "lefigaro":
        base_url = "https://www.lefigaro.fr/flash-actu"
    elif website == "libération":
        base_url = "https://www.liberation.fr/archives/"
    else:
        print("❌ Erreur : site web non reconnu")
        return

    page_number = 0
    headlines: list[str] = []

    # En-têtes HTTP pour simuler un navigateur classique (User-Agent)
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:136.0) Gecko/20100101 Firefox/136.0"
    }

    while len(headlines) < target_data_count:
        # Pause aléatoire entre chaque requête pour simuler un comportement de visiteur humain
        sleep_time = random.uniform(1.0, 2.5)
        time.sleep(sleep_time)

        page_number += 1
        # Construction dynamique de l'URL en fonction du numéro de page
        if website == "legorafi" or website == "nordpresse":
            url = base_url if page_number == 1 else f"{base_url}page/{page_number}/"
        elif website == "lefigaro":
            url = base_url if page_number == 1 else f"{base_url}?page={page_number}"
        elif website == "libération":
            if page_number == 1:
                current_date = datetime.date.today()
            else:
                current_date -= datetime.timedelta(days=1)
            current_date_reformat = current_date.__str__().replace("-", "/")
            url = f"{base_url}{current_date_reformat}/"

        try:
            # Envoi de la requête HTTP avec timeout en cas de souci de connexion
            response = requests.get(url, headers=headers, timeout=5)
            if response.status_code != 200:
                print(
                    f"⚠️ Erreur {response.status_code} : accès impossible à la page {page_number}"
                )
                continue

            # Parsing HTML avec BeautifulSoup
            soup = BeautifulSoup(response.text, "html.parser")

            # Parcourt, extrait et stocke les titres de la page courante
            if website == "legorafi":
                articles = soup.find_all("li", class_="mvp-blog-story-wrap")
                for article in articles:
                    title_tag = article.find("h2")
                    if title_tag:
                        title = title_tag.get_text(strip=True)
                        # Exclusion des titres trop peu substantiels par filtrage de longueur
                        if isinstance(title, str) and len(title) > 10:
                            headlines.append(title)
            elif website == "nordpresse":
                articles = soup.select(
                    "div.tdb_module_loop.td_module_wrap.td-animation-stack.td-cpt-post"
                )
                for article in articles:
                    title_tag = article.find("h3")
                    if title_tag:
                        title = title_tag.find("a").get_text(strip=True)
                        # Exclusion des titres trop peu substantiels par filtrage de longueur
                        if isinstance(title, str) and len(title) > 10:
                            headlines.append(title)
            elif website == "lefigaro":
                title_tags = soup.find_all("h2", class_="fig-flash__title")
                for title_tag in title_tags:
                    title = title_tag.get_text(strip=True)
                    # Exclusion des titres trop peu substantiels par filtrage de longueur
                    if isinstance(title, str) and len(title) > 10:
                        headlines.append(title)
            elif website == "libération":
                title_tags = soup.select("article h2")
                for title_tag in title_tags:
                    title = title_tag.get_text(strip=True)
                    # Exclusion des titres trop peu substantiels par filtrage de longueur
                    if isinstance(title, str) and len(title) > 10:
                        headlines.append(title)

            # Affiche la progression courante par page
            print(f"Page {page_number} — total titres collectés : {len(headlines)}")

        except Exception as e:
            print(f"⚠️ Erreur sur la page {page_number} : {e}")

    print(f"✅ Scraping terminé : {len(headlines)} titres collectés.")
    return headlines

In [ ]:
legorafi_headlines = scrape("legorafi", 3000)

df_legorafi = pd.DataFrame(legorafi_headlines, columns=["headline"])
df_legorafi["is_satirical"] = 1
df_legorafi.to_csv(f"{target_data_dir}/legorafi_headlines.csv", index=False)

Page 1 — total titres collectés : 4
Page 2 — total titres collectés : 8
Page 3 — total titres collectés : 12
Page 4 — total titres collectés : 16
Page 5 — total titres collectés : 20
Page 6 — total titres collectés : 24
Page 7 — total titres collectés : 28
Page 8 — total titres collectés : 32
Page 9 — total titres collectés : 36
Page 10 — total titres collectés : 40
Page 11 — total titres collectés : 44
Page 12 — total titres collectés : 48
Page 13 — total titres collectés : 52
Page 14 — total titres collectés : 56
Page 15 — total titres collectés : 60
Page 16 — total titres collectés : 64
Page 17 — total titres collectés : 68
Page 18 — total titres collectés : 72
Page 19 — total titres collectés : 76
Page 20 — total titres collectés : 80
Page 21 — total titres collectés : 84
Page 22 — total titres collectés : 88
Page 23 — total titres collectés : 92
Page 24 — total titres collectés : 96
Page 25 — total titres collectés : 100
Page 26 — total titres collectés : 104
Page 27 — total titre

In [ ]:
nordpresse_headlines = scrape("nordpresse", 2000)

df_nordpresse = pd.DataFrame(nordpresse_headlines, columns=["headline"])
df_nordpresse["is_satirical"] = 1
df_nordpresse.to_csv(f"{target_data_dir}/nordpresse_headlines.csv", index=False)

Page 1 — total titres collectés : 80
Page 2 — total titres collectés : 160
Page 3 — total titres collectés : 240
Page 4 — total titres collectés : 320
Page 5 — total titres collectés : 400
Page 6 — total titres collectés : 480
Page 7 — total titres collectés : 560
Page 8 — total titres collectés : 640
Page 9 — total titres collectés : 720
Page 10 — total titres collectés : 800
Page 11 — total titres collectés : 880
Page 12 — total titres collectés : 960
Page 13 — total titres collectés : 1039
Page 14 — total titres collectés : 1119
Page 15 — total titres collectés : 1199
Page 16 — total titres collectés : 1279
Page 17 — total titres collectés : 1359
Page 18 — total titres collectés : 1439
Page 19 — total titres collectés : 1519
Page 20 — total titres collectés : 1599
Page 21 — total titres collectés : 1679
Page 22 — total titres collectés : 1758
Page 23 — total titres collectés : 1838
Page 24 — total titres collectés : 1918
Page 25 — total titres collectés : 1998
Page 26 — total titres

In [ ]:
lefigaro_headlines = scrape("lefigaro", 2500)

df_lefigaro = pd.DataFrame(lefigaro_headlines, columns=["headline"])
df_lefigaro["is_satirical"] = 0
df_lefigaro.to_csv(f"{target_data_dir}/lefigaro_headlines.csv", index=False)

Page 1 — total titres collectés : 69
Page 2 — total titres collectés : 138
Page 3 — total titres collectés : 207
Page 4 — total titres collectés : 276
Page 5 — total titres collectés : 345
Page 6 — total titres collectés : 414
Page 7 — total titres collectés : 483
Page 8 — total titres collectés : 552
Page 9 — total titres collectés : 621
Page 10 — total titres collectés : 690
Page 11 — total titres collectés : 759
Page 12 — total titres collectés : 828
Page 13 — total titres collectés : 897
Page 14 — total titres collectés : 966
Page 15 — total titres collectés : 1035
Page 16 — total titres collectés : 1104
Page 17 — total titres collectés : 1173
Page 18 — total titres collectés : 1242
Page 19 — total titres collectés : 1311
Page 20 — total titres collectés : 1380
Page 21 — total titres collectés : 1449
Page 22 — total titres collectés : 1518
Page 23 — total titres collectés : 1587
Page 24 — total titres collectés : 1656
Page 25 — total titres collectés : 1725
Page 26 — total titres c

In [ ]:
liberation_headlines = scrape("libération", 2500)

df_liberation = pd.DataFrame(liberation_headlines, columns=["headline"])
df_liberation["is_satirical"] = 0
df_liberation.to_csv(f"{target_data_dir}/liberation_headlines.csv", index=False)

⚠️ Erreur 404 : accès impossible à la page 1
Page 2 — total titres collectés : 57
Page 3 — total titres collectés : 142
Page 4 — total titres collectés : 225
Page 5 — total titres collectés : 325
Page 6 — total titres collectés : 417
Page 7 — total titres collectés : 494
Page 8 — total titres collectés : 548
Page 9 — total titres collectés : 592
Page 10 — total titres collectés : 684
Page 11 — total titres collectés : 762
Page 12 — total titres collectés : 854
Page 13 — total titres collectés : 935
Page 14 — total titres collectés : 999
Page 15 — total titres collectés : 1045
Page 16 — total titres collectés : 1092
Page 17 — total titres collectés : 1168
Page 18 — total titres collectés : 1249
Page 19 — total titres collectés : 1327
Page 20 — total titres collectés : 1419
Page 21 — total titres collectés : 1501
Page 22 — total titres collectés : 1542
Page 23 — total titres collectés : 1595
Page 24 — total titres collectés : 1686
Page 25 — total titres collectés : 1763
Page 26 — total t